# 案例1：基于TF-IDF的关键词提取
## 开发步骤如下：
> * Step 1：原始数据预处理
> * Step 2：产生IDF词表
> * Step 3：提取句子的关键词

In [1]:
# Step 1
import os
import sys
import math
  
file_path_dir = './d0101/data'
raw_path = './d0101/raw.data'
idf_path = './d0101/idf.data'

def read_file_handler(f):
    fd = open(f, 'r', encoding='utf-8')
    return fd

file_raw_out = open(raw_path, 'w', encoding='utf-8')

# 遍历整个原始数据目录，将零散的文章整合到一个文件中，便于后续数据处理
file_name = 0
for fd in os.listdir(file_path_dir):
    file_path = file_path_dir + '/' + fd

    content_list = []

    file_fd = read_file_handler(file_path)
    for line in file_fd:
        content_list.append(line.strip())

    content = '\t'.join([str(file_name), ' '.join(content_list)]) + '\n'
    file_raw_out.writelines(content)
    
    file_name += 1

file_raw_out.close()

In [2]:
# Step 2
docs_cnt = file_name
wc_tulist = []

with open(raw_path, 'r', encoding='utf-8') as fd:
    for line in fd:
        # 遍历每一篇文章，文章=line
        ss = line.strip().split('\t')
        if len(ss) != 2:
            continue
        # 对文章的解析，区分出文章的名字和文章的内容   
        file_name, file_content = ss
        # 对文章的内容进行切词，因为内容已经按“ ”空格区分好了，所以直接按空格做split就好
        word_list = file_content.strip().split(' ')
        
        # 去重：对于idf，只关心词有没有出现在文章中，至于出现多少次，并不关心
        word_set = set(word_list)

        for word in word_set:
            # 对于每个关键词，打一个标记“1”，来标识该次出现过
            wc_tulist.append((word, '1'))

# 将内容输出到指定目标文件中去
file_idf_out = open(idf_path, 'w', encoding='utf-8')

# 按照词的字典序，进行排序
wc_sort_tulist = sorted(wc_tulist, key=lambda x: x[0])

current_word = None
sum = 0
for tu in wc_sort_tulist:  
    word, val = tu

    if current_word == None:
        current_word = word

    if current_word != word:
        # 通过idf计算公式，得到每个关键词的idf score
        idf = math.log(float(docs_cnt) / (float(sum) + 1.0))
        content = '\t'.join([current_word, str(idf)]) + '\n'
        file_idf_out.write(content)
        current_word = word
        sum = 0

    sum += int(val)

idf = math.log(float(docs_cnt) / (float(sum) + 1.0))
content = '\t'.join([current_word, str(idf)]) + '\n'
file_idf_out.write(content)

file_idf_out.close()


In [3]:
# Step 3
input_str = '我们 带来 阿里巴巴 希望 差'

token_idf_dict = {}
# 将idf字典加载到内存
with open(idf_path, 'r', encoding='utf-8') as fd:
    for line in fd:
        ss = line.strip().split('\t')
        if len(ss) != 2:
            continue
        token, idf_score = ss
        token_idf_dict[token] = idf_score

def get_tfidf(input_str):
    token_dict = {}
    # 对输入字符串的每一个词，计算tf
    for t in input_str.strip().split(' '):
        if t not in token_dict:
            token_dict[t] = 1
        else:
            token_dict[t] += 1

    # res_tu_list = []
    for k, v in token_dict.items():
        tf_score = token_dict[k]
        if k not in token_idf_dict:
            continue
        idf_score = token_idf_dict[k]
        tf_idf = tf_score * float(idf_score)
        yield (k, tf_idf)

for k, v in get_tfidf(input_str):
    print(k, v)


我们 1.0750817912579156
带来 1.4518916436057039
阿里巴巴 2.8552804380848946
希望 1.9321168269229776
差 2.1795250002368185


# 案例2：相似度公式
## 开发步骤如下：
> * Step 1：Cosine
> * Step 2：Jaccard

In [4]:
# Step 1: Cosine

input1_str = '我们 带来 阿里巴巴 希望 差 差 差'
# input2_str = '我们 带来 阿里巴巴 好 好 好'
# input2_str = '我们 带来 搜狐 好 好 好'
input2_str = '我们 带来 阿里巴巴 希望 差 差 差'

def cosine(input1_str, input2_str):
    t1_dict = {}
    sum = 0.
    for k, v in get_tfidf(input1_str):
        sum += pow(v, 2)
    sum = math.sqrt(sum)
    for k, v in get_tfidf(input1_str):
        t1_dict[k] = float(v / sum)

    sum = 0.
    for k, v in get_tfidf(input2_str):
        sum += pow(v, 2)
    sum = math.sqrt(sum)
    
    final_score = 0.
    for k, v in get_tfidf(input2_str):
        if k not in t1_dict:
            continue
        s1 = t1_dict[k]
        s2 = float(v / sum)
        
        final_score += s1 * s2
    return final_score
        
print(cosine(input1_str, input2_str))

1.0


In [5]:
# Step 1: Jaccard
input1_str = '我们 带来 阿里巴巴 希望 差 差 差'
# input2_str = '我们 带来 阿里巴巴 好 好 好'
input2_str = '我们 带来'

def jaccard(input1_str, input2_str):
    s1_set = set(input1_str.strip().split(' '))
    s2_set = set(input2_str.strip().split(' '))
    
    score = 0.
    s1_s2_join = s1_set & s2_set
    len1 = len(s1_s2_join)
    
    s1_s2_union = s1_set | s2_set
    len2 = len(s1_s2_union)
    
    return float(len1) / float(len2)
    
print(jaccard(input1_str, input2_str))

0.4
